In [1]:
from abc import ABC, abstractmethod
from textwrap import dedent


class LLMPipe(ABC):
    """Abstract class for taking input and outputting a message"""
    @abstractmethod
    def __call__(self, prompt: str) -> str:
        """Take prompt and output a message."""
        pass

class OllamaLLM(LLMPipe):
    """Ollama Language Model"""
    def __init__(self, model_name: str, temperature: float):
        from langchain.llms.ollama import Ollama
        self._model = Ollama(model=model_name, temperature=temperature)

    def __call__(self, prompt: str) -> str:
        return self._model(prompt)


def divide(number: float, divisor: float) -> str:
    return str(float(number) / float(divisor))

prompt = dedent(
    """
    Instructions:
    You are an AI agent capable of taking actions in sequence (steps) until you achieve the desired goal stated by the user.
    Each step, you will take on action from the following list:
        - divide(number: float, divisor: float) -> str
        - return_final_request(value: str)
    When you are finished, call the return_final_request(value: str) function to finish the program.
    
    User:
    I want to divide 5 by 3.

    Previous Steps:
    {previous_steps}

    Next Action (Just type the function name and arguments):
    """
)

model = OllamaLLM(model_name="mistral", temperature=0)
previous_steps = [] # stack of (action, result)
def format_previous_steps(s):
    if not s:
        return "N/A"
    return "\n".join(f"{action} -> {result}" for action, result in s)

while True:
    prompt = prompt.format(previous_steps=format_previous_steps(previous_steps))
    print(f"Prompt: {prompt}")
    action = model(prompt).strip()
    
    if action.startswith("divide("):
        number, divisor = action[7:-1].split(",")
        result = divide(number, divisor)
        previous_steps.append((action, result))
        print(f"Result: {result}")
        print(f"Previous Steps: {previous_steps}")



Prompt: 
Instructions:
You are an AI agent capable of taking actions in sequence (steps) until you achieve the desired goal stated by the user.
Each step, you will take on action from the following list:
    - divide(number: float, divisor: float) -> str
    - return_final_request(value: str)
When you are finished, call the return_final_request(value: str) function to finish the program.

User:
I want to divide 5 by 3.

Previous Steps:
N/A

Next Action (Just type the function name and arguments):

Result: 1.6666666666666667
Previous Steps: [('divide(5, 3)', '1.6666666666666667')]
Prompt: 
Instructions:
You are an AI agent capable of taking actions in sequence (steps) until you achieve the desired goal stated by the user.
Each step, you will take on action from the following list:
    - divide(number: float, divisor: float) -> str
    - return_final_request(value: str)
When you are finished, call the return_final_request(value: str) function to finish the program.

User:
I want to divid

KeyboardInterrupt: 

: 